In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image


model = load_model('/mnt/d/Coding/tensorflowProjects/tf217/final_model.keras')
class_labels = {0: 'DR', 1: 'No_DR'}


def preprocess_image(image):
    img_array = np.array(image)
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    filtered = cv2.GaussianBlur(gray, (5, 5), 0)
    _, segmented = cv2.threshold(filtered, 30, 255, cv2.THRESH_BINARY)
    return gray, filtered, segmented


def predict_image(image):
    img = image.resize((224, 224))
    img_array = np.array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))

    
    prediction = model.predict(img_array)
    predicted_class = int(prediction[0, 0] > 0.5)
    confidence = prediction[0, 0] if predicted_class == 1 else 1 - prediction[0, 0]
    diagnosis = class_labels[predicted_class]
    
    
    result_text = f"Diagnosis: {diagnosis} (Confidence: {confidence:.2f})"
    return result_text, (diagnosis == "DR" and "red") or "green"


def process_and_predict(image):
    gray, filtered, segmented = preprocess_image(image)
    result_text, result_color = predict_image(image)
    
    return (
        result_text,
        result_color,
        image,       # Original Image
        gray,        # Gray Image
        filtered,    # Filtered Image
        segmented    # Segmented Image
    )


interface = gr.Interface(
    fn=process_and_predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Detection Result"),
        gr.Label(label="Confidence Level", color="dynamic"),
        gr.Image(label="Original Image"),
        gr.Image(label="Gray Image"),
        gr.Image(label="Filtered Image"),
        gr.Image(label="Segmented Image"),
    ],
    title="Diabetic Retinopathy Detection",
    description="Upload a retinal image to detect the presence and severity of Diabetic Retinopathy."
)

if __name__ == "__main__":
    interface.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
